conda install openpyxl

In [109]:
import pandas as pd  
from datetime import datetime, timedelta
import re
import os
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words

In [110]:
#กำหนด input file / ชื่อ output file
input='RawDataCSV\Biore UV Aqua Rich Watery Gel_20240922_153520.csv'
output='Biore UV Aqua Rich Watery Gel_20240922_153520.csv'

<>:2: SyntaxWarning: invalid escape sequence '\B'
<>:2: SyntaxWarning: invalid escape sequence '\B'
C:\Users\earns\AppData\Local\Temp\ipykernel_10720\2128643595.py:2: SyntaxWarning: invalid escape sequence '\B'
  input='RawDataCSV\Biore UV Aqua Rich Watery Gel_20240922_153520.csv'


In [111]:
def convert_date(date_str, file_date):
    # แปลงวันที่จากชื่อไฟล์
    file_date = datetime.strptime(file_date, '%Y%m%d')
    
    # กรณีที่มีคำว่า 'ก่อน'
    if 'ก่อน' in date_str:
        value, unit = date_str.split()[:2]
        value = int(value)
        
        if unit in ['วินาทีก่อน', 'นาทีก่อน', 'ชั่วโมงก่อน']:
            return file_date.strftime('%Y%m%d')
        elif unit == 'วันก่อน':
            return (file_date - timedelta(days=value)).strftime('%Y-%m-%d')
        elif unit == 'สัปดาห์ก่อน':
            return (file_date - timedelta(weeks=value)).strftime('%Y-%m-%d')
    
    month_dict = {
        'ม.ค.': '01', 'ก.พ.': '02', 'มี.ค.': '03', 'เม.ย.': '04',
        'พ.ค.': '05', 'มิ.ย.': '06', 'ก.ค.': '07', 'ส.ค.': '08',
        'ก.ย.': '09', 'ต.ค.': '10', 'พ.ย.': '11', 'ธ.ค.': '12'
    }
    
    parts = date_str.split()
    if len(parts) == 3 and parts[1] in month_dict:
        day, month, year = parts
        return f"{year}-{month_dict[month]}-{int(day):02d}"
    
    return date_str

def process_dataframe(file_path):
    # อ่านไฟล์ CSV
    df = pd.read_csv(file_path)
    
    # แยกวันที่จากชื่อไฟล์
    file_date = file_path.split('_')[-2]
    
    # แปลงคอลัมน์ date
    df['date'] = df['date'].apply(lambda x: convert_date(x, file_date))
    
    return df


In [112]:
df = process_dataframe(input)
print(df)

      no                        shop         name  rating        date  \
0      1  Kao Beauty & Personal Care        จ***.       5  2024-08-25   
1      2  Kao Beauty & Personal Care        P***i       5  2024-06-23   
2      3  Kao Beauty & Personal Care     นิตยา เ.       5  2024-08-15   
3      4  Kao Beauty & Personal Care        g***t       5  2024-07-18   
4      5  Kao Beauty & Personal Care  พิณพินัช บ.       5  2024-06-04   
..   ...                         ...          ...     ...         ...   
405  406                     Lotus’s   Kawduan N.       4  2021-04-27   
406  407                     Lotus’s   สุรวุฒิ แ.       4  2017-03-25   
407  408                     Lotus’s   มนัสชนก ก.       4  2019-05-29   
408  409                     Lotus’s     Pocky P.       4  2018-04-24   
409  410                     Lotus’s     แกรี่ ว.       5  2024-04-22   

                                           review_text  
0    ร้านค้าจัดส่งสินค้าได้รวดเร็วมาก การแพ็คสินค้า...  
1    สะใจ

In [113]:
df.dtypes

no              int64
shop           object
name           object
rating          int64
date           object
review_text    object
dtype: object

In [114]:
# ฟังก์ชันลบข้อมูลที่ไม่ใช่ภาษาไทย
def clean_review(text):
    # ลบอีโมจิ, ภาษาอังกฤษ, ตัวเลข, สัญลักษณ์พิเศษ และช่องว่างเกินจำเป็น
    cleaned_text = re.sub(r'[^\u0E00-\u0E7F\s]', '', text)  # \u0E00-\u0E7F คือตัวอักษรภาษาไทยใน Unicode
    return cleaned_text.strip()

# ใช้ฟังก์ชัน clean_review กับคอลัมน์ review_text
df['review_text'] = df['review_text'].apply(lambda x: clean_review(str(x)))

In [115]:
# ลบการขึ้นบรรทัดใหม่ใน review_text และทำให้ข้อความอยู่ในบรรทัดเดียวกัน
df['review_text'] = df['review_text'].apply(lambda x: str(x).replace('\n', ' ').strip())

In [116]:
# โหลดฐานข้อมูลคำพื้นฐาน
word_list = set(thai_words())

def is_meaningful(word):
    # เพิ่มการตรวจสอบคำที่มีรูปแบบพิเศษ
    special_patterns = [
        r'^[ก-ฮ]+$',  # คำภาษาไทยทั่วไป
        r'^[ก-ฮ]+ๆ$',  # คำซ้ำ เช่น ดีๆ เร็วๆ
        r'^[ก-ฮ]+[ุึ][ปบ]$',  # คำลงท้ายด้วย ุป, ึป, ุบ, ึบ เช่น ปุ๊ป, ปั๊บ
        r'^แพ[็ค]$',  # คำว่า แพ็ค หรือ แพค
        r'^[ก-ฮ]{1,2}่[ก-ฮ]$',  # คำสั้นๆ ที่มีวรรณยุกต์ เช่น ค่ะ ไว
        r'^[0-9๐-๙]+$',  # ตัวเลข
        r'^[a-zA-Z]+$',  # คำภาษาอังกฤษ
    ]
    
    return (word in word_list or
            any(re.match(pattern, word) for pattern in special_patterns))

def clean_text(text):
    # แยกคำ
    words = word_tokenize(text)
    
    # แทนที่คำที่ไม่มีความหมายด้วยช่องว่าง
    cleaned_words = [word if is_meaningful(word) else ' ' for word in words]
    
    # รวมคำกลับเป็นประโยค
    cleaned_text = ''.join(cleaned_words)
    
    # ลบช่องว่างซ้ำ
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # ลบช่องว่างที่ไม่จำเป็นตอนต้นและท้ายข้อความ
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

In [117]:
df['review_text'] = df['review_text'].apply(clean_text)

In [118]:
# ลบแถวที่ review_text ว่าง (ที่ไม่มีข้อมูลภาษาไทยหรือเป็นช่องว่าง)
df_cleaned = df[df['review_text'].astype(bool)].copy()

In [119]:
df_cleaned

,no,shop,name,rating,date,review_text
0,1,Kao Beauty & Personal Care,จ***.,5,2024-08-25,ร้านค้าจัดส่งสินค้าได้รวดเร็วมาก การแพ็คสินค้า...
1,2,Kao Beauty & Personal Care,P***i,5,2024-06-23,สะใจมากกันแดดขวดนี้ปริมาณ กดมาได้ในราคา บาท รา...
2,3,Kao Beauty & Personal Care,นิตยา เ.,5,2024-08-15,สูตรที่ยาวนาน อ่อนโยนต่อผิวแพ้ง่าย รู้สึกสดชื่...
3,4,Kao Beauty & Personal Care,g***t,5,2024-07-18,สินค้า คุณภาพดีสมราคาดีเลย การจัดส่ง ไม่ช้ามาก...
4,5,Kao Beauty & Personal Care,พิณพินัช บ.,5,2024-06-04,อ่อนโยนต่อผิวที่แพ้ง่าย ซึมเข้าสู่ผิวได้อย่างร...
...,...,...,...,...,...,...
402,403,Lotus’s,จิรัชญา ส.,4,2019-10-16,กันแดดเนื้อเป็นน้ำเจล ทาตอนแรกจะออกวอก ขาว ต้อ...
404,405,Lotus’s,K***k,4,2023-10-03,คุณภาพ โดยรวมถือว่าโอเคค่ะใช้ได้ดีอยากให้มีโปร...
405,406,Lotus’s,Kawduan N.,4,2021-04-27,สภาพสินค้าดีมากค่ะไม่มีที่ติ แต่ของรอนานมากค่ะ
406,407,Lotus’s,สุรวุฒิ แ.,4,2017-03-25,ใช้ดี ราคาไม่แพงครับ ถูกกว่าซื้อเองตามร้าน ติน...


In [120]:
# รีเซ็ตเลขลำดับคอลัมน์ 'no' ใน DataFrame ที่ผ่านการตรวจสอบ
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned['no'] = df_cleaned.index + 1

In [121]:
df_cleaned

,no,shop,name,rating,date,review_text
0,1,Kao Beauty & Personal Care,จ***.,5,2024-08-25,ร้านค้าจัดส่งสินค้าได้รวดเร็วมาก การแพ็คสินค้า...
1,2,Kao Beauty & Personal Care,P***i,5,2024-06-23,สะใจมากกันแดดขวดนี้ปริมาณ กดมาได้ในราคา บาท รา...
2,3,Kao Beauty & Personal Care,นิตยา เ.,5,2024-08-15,สูตรที่ยาวนาน อ่อนโยนต่อผิวแพ้ง่าย รู้สึกสดชื่...
3,4,Kao Beauty & Personal Care,g***t,5,2024-07-18,สินค้า คุณภาพดีสมราคาดีเลย การจัดส่ง ไม่ช้ามาก...
4,5,Kao Beauty & Personal Care,พิณพินัช บ.,5,2024-06-04,อ่อนโยนต่อผิวที่แพ้ง่าย ซึมเข้าสู่ผิวได้อย่างร...
...,...,...,...,...,...,...
276,277,Lotus’s,จิรัชญา ส.,4,2019-10-16,กันแดดเนื้อเป็นน้ำเจล ทาตอนแรกจะออกวอก ขาว ต้อ...
277,278,Lotus’s,K***k,4,2023-10-03,คุณภาพ โดยรวมถือว่าโอเคค่ะใช้ได้ดีอยากให้มีโปร...
278,279,Lotus’s,Kawduan N.,4,2021-04-27,สภาพสินค้าดีมากค่ะไม่มีที่ติ แต่ของรอนานมากค่ะ
279,280,Lotus’s,สุรวุฒิ แ.,4,2017-03-25,ใช้ดี ราคาไม่แพงครับ ถูกกว่าซื้อเองตามร้าน ติน...


In [23]:
if not os.path.exists('CleanDataCSV'):
    os.makedirs('CleanDataCSV')
if not os.path.exists('CleanDataExel'):
    os.makedirs('CleanDataExel')

In [122]:
# บันทึก DataFrame ลงไฟล์ CSV
csv_file = f'CleanDataCSV/{output}.csv'
df_cleaned.to_csv(csv_file, index=False)

# บันทึก DataFrame ลงไฟล์ Excel
excel_file = f'CleanDataExel/{output}.xlsx'
df_cleaned.to_excel(excel_file, index=False)

In [1]:
import re
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus.common import thai_words
from pythainlp.spell import correct

# โหลดฐานข้อมูลคำพื้นฐาน
word_list = set(thai_words())

def is_meaningful(word):
    # เพิ่มการตรวจสอบคำที่มีรูปแบบพิเศษ
    special_patterns = [
        r'^[ก-ฮ]+$',  # คำภาษาไทยทั่วไป
        r'^[ก-ฮ]+ๆ$',  # คำซ้ำ เช่น ดีๆ เร็วๆ
        r'^[ก-ฮ]+[ุึ][ปบ]$',  # คำลงท้ายด้วย ุป, ึป, ุบ, ึบ เช่น ปุ๊ป, ปั๊บ
        r'^แพ[็ค]$',  # คำว่า แพ็ค หรือ แพค
        r'^[ก-ฮ]{1,2}่[ก-ฮ]$',  # คำสั้นๆ ที่มีวรรณยุกต์ เช่น ค่ะ ไว
        r'^[0-9๐-๙]+$',  # ตัวเลข
        r'^[a-zA-Z]+$',  # คำภาษาอังกฤษ
    ]
    
    return (word in word_list or
            any(re.match(pattern, word) for pattern in special_patterns))

def clean_text(text):
    # แยกคำ
    words = word_tokenize(text)
    
    # แทนที่คำที่ไม่มีความหมายด้วยช่องว่าง
    cleaned_words = [word if is_meaningful(word) else ' ' for word in words]
    
    # รวมคำกลับเป็นประโยค
    cleaned_text = ''.join(cleaned_words)
    
    # ลบช่องว่างซ้ำ
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # ลบช่องว่างที่ไม่จำเป็นตอนต้นและท้ายข้อความ
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

# ตัวอย่างการใช้งาน
examples = [
    "ใช้งานดีมากค่ะ กันแดดเนื้อเจล ปาดปุ๊ปซึมปั๊ปปปปปปป",
    "ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึมเร็ว ไม่เหนียวเหมาะสำหรับใช้ในชีวิตประจำวัน",
    "นใำนฝำนฝพนฝกนฝดสฝดนฝะัตงพนฝะนฝพนพนใำนใพนฝำนวำนฝดรใดรใด่มดรมำรมำรใพรใพนใพนฝพนฝพนฝพนฝดนใพน",
    "ดีมากๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ",
    "ั  ชอบมาก ถูกมาก ดีมาก คุ้มค่า คุ้มราคาสุดๆ",
    "วแ่วแว่แว่อทใ ทใว่แ่วแีวแว่ งาอยีดัวแฝท  าฃอ่วแง่ งา ว่แีววีแวาแ งา ง่แางอางดีงแาง สฃอบรแยีแงาอางอบรเหมาะสำหรับกิจกรรมกลางแจ้ง, เหมาะสำหรับการใช้งานทุกวัน,"
]

for example in examples:
    print("Original:", example)
    print("Cleaned:", clean_text(example))
    print()

# สำหรับการใช้งานกับ DataFrame
# df['cleaned_review'] = df['review_text'].apply(clean_text)
# print(df[['review_text', 'cleaned_review']].head())

Original: ใช้งานดีมากค่ะ กันแดดเนื้อเจล ปาดปุ๊ปซึมปั๊ปปปปปปป
Cleaned: ใช้งานดีมากค่ะ กันแดดเนื้อเจล ปาดปุ๊ปซึมปั๊ปปปปปปป

Original: ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึมเร็ว ไม่เหนียวเหมาะสำหรับใช้ในชีวิตประจำวัน
Cleaned: ส่งไว แพคมาอย่างดี ไม่มีเสียหาย ลองใช้ดูแล้วซึมเร็ว ไม่เหนียวเหมาะสำหรับใช้ในชีวิตประจำวัน

Original: นใำนฝำนฝพนฝกนฝดสฝดนฝะัตงพนฝะนฝพนพนใำนใพนฝำนวำนฝดรใดรใด่มดรมำรมำรใพรใพนใพนฝพนฝพนฝพนฝดนใพน
Cleaned: ใด

Original: ดีมากๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆๆ
Cleaned: ดีมาก

Original: ั  ชอบมาก ถูกมาก ดีมาก คุ้มค่า คุ้มราคาสุดๆ
Cleaned: ชอบมาก ถูกมาก ดีมาก คุ้มค่า คุ้มราคาสุด

Original: วแ่วแว่แว่อทใ ทใว่แ่วแีวแว่ งาอยีดัวแฝท  าฃอ่วแง่ งา ว่แีววีแวาแ งา ง่แางอางดีงแาง สฃอบรแยีแงาอางอบรเหมาะสำหรับกิจกรรมกลางแจ้ง, เหมาะสำหรับการใช้งานทุกวัน,
Cleaned: งาอยี แง่ งา วี งา อางดี แง อางอบรเหมาะสำหรับกิจกรรมกลางแจ้ง เหมาะสำหรับการใช้งานทุกวัน



In [93]:
from pythainlp.spell import correct

test = correct("ปุ๊ป")
print(test)

ปุ๊ป
